In [ ]:
%matplotlib notebook
import numpy as np
from astropy.table import Table, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
import scipy.stats
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
plt.ion()

knn = 10
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
astorad = np.pi/(180.*60.*60)
Rauto_cut = 22.9
dz_membership = 0.02
Qcut = 3

# Load the classifications and photometry

In [ ]:
classtable = Table().read('../catalogs/classified_objects.csv')
for i in range(knn):
    classtable[str(i+1)+'NN_arcsec'] = np.zeros(len(classtable))

In [ ]:
phottable = Table().read('../catalogs/slits_phot_zs_cutonslitdist_rcl.csv')
phottable = phottable[np.where(phottable['Rauto'] < Rauto_cut)]
phottable = phottable[np.where(phottable['Q'] >= Qcut)]

photuncut = Table().read('../catalogs/slits_phot_zs_cutonslitdist_rcl.csv')

# Find the k-NN (arcsec) and calculate the harmonic mean

In [ ]:
for row in classtable:
    # Find nearby objects in z
    subtable = phottable[np.where(abs(phottable['zLDP'] - row['zLDP']) < dz_membership)]
    subcoords = SkyCoord(ra=subtable['ra']*u.degree, dec=subtable['dec']*u.degree)
    

    obj = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree)
    seps = subcoords.separation(obj).arcsecond
    kseps = sorted(seps)[1:knn+1]
    for i in range(knn):
        row[str(i+1)+'NN_arcsec'] = kseps[i]
    

# Calculate density in galaxies/Mpc$^2$

In [ ]:
classtable['harm_mean'] = np.zeros(len(classtable))
classtable['surface_density'] = np.zeros(len(classtable))

for row in classtable:
    DA = cosmo.angular_diameter_distance(row['zLDP'])
    kseps = np.array([row[str(i+1)+'NN_arcsec'] for i in range(knn)]) * astorad * DA.to('Mpc').value
    row['harm_mean'] = scipy.stats.hmean(kseps)
    
    # Account for the object itself!
    n_in_area = len(np.where(kseps<row['harm_mean'])[0]) + 1
    row['surface_density'] = n_in_area/(np.pi*row['harm_mean']*row['harm_mean'])

# Make the completeness correction
At the moment, this is to just divide by the Q completeness at the relevent radius.

In [ ]:
completeness = Table().read('../catalogs/radial_completeness.dat', format='ascii')
completeness['theta_cl_min'] = completeness['theta_cl'] - 0.25
completeness['theta_cl_max'] = completeness['theta_cl'] + 0.25

In [ ]:
classtable['surface_density_compcorred'] = np.zeros(len(classtable))
for row in classtable:
    rcl = (row['theta_cl_radian']*u.radian).to(u.arcminute).value
    for crow in completeness:
        if crow['theta_cl_min'] < rcl < crow['theta_cl_max']:
            row['surface_density_compcorred'] = row['surface_density']/crow['q'+str(Qcut)]
            break

# Save the updated table

In [ ]:
#classtable.write('../catalogs/classified_surfacedensity.csv', format='csv', overwrite=True)

# Repeat for the cluster cores

In [ ]:
def make_subcat(fullcat, intflag, flag):
    subcat = fullcat[np.where(fullcat['morph_int'].astype(int) == intflag)]
    subcat['morph'] = flag
    return subcat

In [ ]:
# Load cluster core classifications
desai = Table(np.loadtxt('../catalogs/desai07_classifiedGalaxies.cat', skiprows=8, usecols=[2,3,5]),
             names=['ra', 'dec', 'morph_int'])
desai['morph'] = 'xxx'

# Cut to only morphs I care about
allowed_morphs = [[-5, 'E'], [-2, 'S0'], [1, 'Sa'], [2, 'Sa'], [3, 'Sb'], [4, 'Sb'],
                  [5, 'Sc'], [6, 'Sc'], [7, 'Sd'], [8, 'Sd'], [9, 'Sm'], [10, 'Sm'], [11, 'Irr']]   
desaiclass = vstack([make_subcat(desai, morph[0], morph[1]) for morph in allowed_morphs])

# Match to the ediscs catalogs
photcoords = SkyCoord(ra=photuncut['ra']*u.degree, dec=photuncut['dec']*u.degree)
desaicoords = SkyCoord(ra=desaiclass['ra']*u.degree, dec=desaiclass['dec']*u.degree)
idx, d2d, d3d = desaicoords.match_to_catalog_sky(photcoords)
d2d = d2d.to(u.arcsec).value

# Copy over zLDP, Q, and 2d distance
desaiclass['sep_arcsec'] = d2d
desaiclass['zLDP'] = photuncut[idx]['zLDP']
desaiclass['Q'] = photuncut[idx]['Q']
desaiclass['Rauto'] = photuncut[idx]['Rauto']
desaiclass['theta_cl_radian'] = photuncut[idx]['theta_cl_radian']

# Apply cuts: 2dsep < 5as, Qcut, Rauto_cut
desaiclass = desaiclass[desaiclass['Q'] >= Qcut]
desaiclass = desaiclass[desaiclass['Rauto'] < Rauto_cut]
desaiclass = desaiclass[desaiclass['sep_arcsec'] < 5.0]

# Add some rows to the table
for i in range(knn):
    desaiclass[str(i+1)+'NN_arcsec'] = np.zeros(len(desaiclass))
desaiclass['harm_mean'] = np.zeros(len(desaiclass))
desaiclass['surface_density'] = np.zeros(len(desaiclass))
desaiclass['surface_density_compcorred'] = np.zeros(len(desaiclass))

# Find the k NN
for row in desaiclass:
    # Find nearby objects in z
    subtable = phottable[np.where(abs(phottable['zLDP'] - row['zLDP']) < dz_membership)]
    subcoords = SkyCoord(ra=subtable['ra']*u.degree, dec=subtable['dec']*u.degree)
    
    obj = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree)
    seps = subcoords.separation(obj).arcsecond
    kseps = sorted(seps)[1:knn+1]
    for i in range(knn):
        row[str(i+1)+'NN_arcsec'] = kseps[i]

# Calculate the surface density within the harmonic mean of the k NN
for row in desaiclass:
    DA = cosmo.angular_diameter_distance(row['zLDP'])
    kseps = np.array([row[str(i+1)+'NN_arcsec'] for i in range(knn)]) * astorad * DA.to('Mpc').value
    row['harm_mean'] = scipy.stats.hmean(kseps)
    
    # Account for the object itself!
    n_in_area = len(np.where(kseps<row['harm_mean'])[0]) + 1
    row['surface_density'] = n_in_area/(np.pi*row['harm_mean']*row['harm_mean'])

# Make the completeness correction
for row in desaiclass:
    rcl = (row['theta_cl_radian']*u.radian).to(u.arcminute).value
    for crow in completeness:
        if crow['theta_cl_min'] < rcl < crow['theta_cl_max']:
            row['surface_density_compcorred'] = row['surface_density']/crow['q'+str(Qcut)]
            break

# Save the table
desaiclass.write('../catalogs/classified_surfacedensity_desai.csv', format='csv', overwrite=True)